## 二値整数計画問題
連立方程式を満たしたうえである計算を最小化する、制約付きの最小値問題です。  
より正確には、$Sx = b$を満たすベクトル$x$のうち、$cx$を最小にするxを探します

### QUBO
この問題は二つの式から成り立ちます。連立方程式の制約を満たすための$H_A$と最小コストを求めるための$H_B$です。

連立方程式を満たす式
$$
H_A = \sum_{j=1}^m[b_j - \sum_{i=1}^NS_{ji}x_i]^2
$$

ベクトルの計算を最小化する式
$$
H_B = - \sum_{i=1}^Nc_ix_i
$$

つないだ式。つなぐときには調整パラメーターが必要になりますので、$\lambda$を付けました。
$$
H = H_A + \lambda*H_B = \sum_{j=1}^m[b_j - \sum_{i=1}^NS_{ji}x_i]^2 - \lambda*\sum_{i=1}^Nc_ix_i
$$

### 例題

下記を満たすxのうち、

$$
\begin{bmatrix}
3 & 2 & 1 \\
5 & 2 & 3
\end{bmatrix}
\begin{bmatrix}
x_0\\x_1\\x_2
\end{bmatrix}
=
\begin{bmatrix}
3\\5
\end{bmatrix}
$$

次の式を最小にするようなxを求める。

$$
- \begin{bmatrix}
1&2&1
\end{bmatrix}
\begin{bmatrix}
x_0\\x_1\\x_2
\end{bmatrix}
$$

制約条件を満たすとき、QUBO式は0になります。  
そのため、0にならなかったら制約を満たしていないというのがすぐにわかります。  
制約を満たしてなかったらそもそも答えとして成立しません。

In [1]:
from tytan import symbols, Compile, sampler

# 変数を定義 q_0, q_1, q_2
q = symbols("q_{0:3}")

#式を別々に用意します。
HA = (3*q[0]+2*q[1]+q[2]-3)**2 + (5*q[0]+2*q[1]+3*q[2]-5)**2
HB = -(q[0]+2*q[1]+q[2])

#つけるときに今回はConstraint関数を使うことであとで簡単に確かめができます。Constraintは制約条件の方だけにつけます。
M = 10
H = HA + M*HB

# Compileクラスを使用して、QUBOを取得
Q, offset = Compile(H).get_qubo()

print(Q,offset)

# サンプラーを選択
solver = sampler.SASampler()

#クラウドサンプラーの場合
#API_KEY = "API key"
#solver = sampler.NQSSampler()
#result = solver.run(Q, api_key=API_KEY)

# 計算
result = solver.run(Q, shots=100)
print(result)

{('q_{0}', 'q_{0}'): -44.0, ('q_{1}', 'q_{1}'): -44.0, ('q_{2}', 'q_{2}'): -36.0, ('q_{1}', 'q_{2}'): 16.0, ('q_{0}', 'q_{1}'): 32.0, ('q_{0}', 'q_{2}'): 36.0} 34
[[{'q_{0}': 0, 'q_{1}': 1, 'q_{2}': 1}, -64.0, 49], [{'q_{0}': 1, 'q_{1}': 1, 'q_{2}': 0}, -56.0, 51]]


In [2]:
print("Sample =", result[0][0])
print("Cost =", result[0][1] + offset)

Sample = {'q_{0}': 0, 'q_{1}': 1, 'q_{2}': 1}
Cost = -30.0000000000000


011が正解で、コストが-30.0となりました。